### Libraries

In [9]:
import pandas as pd

### Datasets

In [10]:
data_DA = pd.read_csv('datas/GLASSDOOR_DA_LASTMONTH.csv', encoding_errors='ignore')
data_DE = pd.read_csv('datas/GLASSDOOR_DE_LASTMONTH.csv', encoding_errors='ignore')
data_DS = pd.read_csv('datas/GLASSDOOR_DS_LASTMONTH.csv', encoding_errors='ignore')

In [11]:
#Drop the first row
data_DA = data_DA.drop(data_DA.index[0])
data_DS = data_DS.drop(data_DS.index[0])

In [12]:
# append the dataframes
data = data_DA.append(data_DE, ignore_index=True)
data = data.append(data_DS, ignore_index=True)

data.shape

C:\Users\gaspa\AppData\Local\Temp\ipykernel_21652\648892927.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data_DA.append(data_DE, ignore_index=True)
C:\Users\gaspa\AppData\Local\Temp\ipykernel_21652\648892927.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(data_DS, ignore_index=True)


(2366, 10)

In [13]:
data.head()

,companyName,company_starRating,company_offeredRole,company_roleLocation,headquarters,size,type,revenue,listing_jobDesc,requested_url
0,Caisse d'Epargne Hauts de France,3.9,DATA ANALYST (H/F),Lille,"Paris, France",10000+ Employees,Company - Public,Unknown / Non-Applicable,"Une rmunration comptitive et quitable, assorti...",https://www.glassdoor.com/partner/jobListing.h...
1,Natixis,4.1,Business Analyst Junior - Third Parties - IT CIB,Charenton-le-Pont,"Paris, France",10000+ Employees,Company - Private,$5 to $10 billion (USD),\nRecueillir et traduire les expression des be...,https://www.glassdoor.com/partner/jobListing.h...
2,Sopra Steria,3.8,Business Analyst - Banque & Finance - Brest,Brest,"Annecy, France",10000+ Employees,Company - Public,$1 to $5 billion (USD),Dfinition des besoins utilisateurs ou User St...,https://www.glassdoor.com/partner/jobListing.h...
3,HURRYMAN,4.8,Consultant Senior / Manager : Data Analyst (H/F),Paris,"Paris, France",1 to 50 Employees,Company - Private,Unknown / Non-Applicable,uvrant parmi les acteurs les plus prestigieux ...,https://www.glassdoor.com/partner/jobListing.h...
4,Editions ENI,4.0,Data Analyst H/F,Saint-Herblain,"Saint-Herblain, France",201 to 500 Employees,Unknown,$25 to $100 million (USD),Echanges et coordination avec les quipes dcisi...,https://www.glassdoor.com/partner/jobListing.h...


In [14]:
#assigner au hasard une valeur a data["type_poste"] entre Data Analyst, Data Scientist et Data Engineer
import random
import numpy as np

def random_poste():
    return random.choice(["Data Analyst", "Data Scientist", "Data Engineer"])

data["type_poste"] = np.vectorize(random_poste)()

In [15]:
from geopy.geocoders import Nominatim
from geopy.distance import geodesic

def get_coordinates(city):
    geolocator = Nominatim(user_agent="distance_calculator")
    location = geolocator.geocode(city)
    if location is None:
        return None
    return location.latitude, location.longitude

def calculate_distance(city1, city2):
    coordinates1 = get_coordinates(city1)
    coordinates2 = get_coordinates(city2)
    
    if coordinates1 is None or coordinates2 is None:
        return "Impossible de trouver les coordonnées de l'une des villes."
    
    distance = geodesic(coordinates1, coordinates2).kilometers
    return distance

# Exemple d'utilisation
city1 = "Beuzeville-la-bastille"
city2 = "Auray"

distance = calculate_distance(city1, city2)
print(f"La distance entre {city1} et {city2} est d'environ {distance:.2f} kilomètres.")


La distance entre Beuzeville-la-bastille et Auray est d'environ 222.35 kilomètres.


In [16]:
from haversine import haversine, Unit

def calculate_distance(city1, city2):
    coordinates1 = get_coordinates(city1)
    coordinates2 = get_coordinates(city2)
    
    if coordinates1 is None or coordinates2 is None:
        return "Impossible de trouver les coordonnées de l'une des villes."
    
    distance = haversine(coordinates1, coordinates2, unit=Unit.KILOMETERS)
    return distance


In [ ]:
for row in data[1:100].iterrows():
    print(row[1]["city"])
    print(calculate_distance(row[1]["city"], "Paris"))
    print("")

In [17]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from geopy.distance import geodesic

df = data.copy()


# Préférences de l'utilisateur
ville_preferee = 'Paris'
type_entreprise_prefere = 'Start-up'
type_poste_desire = 'Data Analyst'

# Calcul de la distance entre les villes
df['distance_en_km'] = df.apply(lambda row: calculate_distance(row["company_roleLocation"], ville_preferee), axis=1)

# Calcul de la similarité cosinus
vectorizer = TfidfVectorizer()
job_desc = df['company_offeredRole'] + ' ' + df['size'] + ' ' + df['type_poste']
user_preferences = ville_preferee + ' ' + type_entreprise_prefere + ' ' + type_poste_desire
X = vectorizer.fit_transform(job_desc.append(pd.Series(user_preferences)))
similarity_scores = cosine_similarity(X[:-1], X[-1])

# Normalisation de la distance
max_distance = df['distance_en_km'].max()
df['normalized_distance'] = 1 - (df['distance_en_km'] / max_distance)

# Attribution des poids aux similarités et à la distance
similarity_weight = 0.8
distance_weight = 0.2

# Création d'une nouvelle colonne pour stocker les scores de similarité
df['similarity_score'] = similarity_scores.flatten()

# Calcul de la similarité pondérée
df['weighted_similarity'] = df['similarity_score'] * similarity_weight
df['weighted_distance'] = df['normalized_distance'] * distance_weight
df['weighted_similarity_distance'] = df['weighted_similarity'] + df['weighted_distance']

# Tri des offres d'emploi par similarité pondérée et distance croissantes
sorted_df = df.sort_values(by=['weighted_similarity_distance', 'distance_en_km'], ascending=[False, True])

# Affichage des résultats
top_results = sorted_df.head(5)  # Vous pouvez ajuster le nombre de résultats à afficher
top_results[['company_offeredRole', 'companyName', 'distance_en_km', 'weighted_similarity_distance']].head()


KeyboardInterrupt: 

In [ ]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from geopy.distance import geodesic

# Exemple de dataframe simplifié
df = pd.DataFrame({
    'intitule_poste': ['Développeur web', 'Data analyst', 'Ingénieur logiciel'],
    'ville_du_poste': ['Paris', 'Lyon', 'Marseille'],
    'type_entreprise': ['Start-up', 'Grande entreprise', 'Start-up'],
    'nom_entreprise': ['XYZ Company', 'ABC Corporation', 'XYZ Company'],
    'latitude': [48.8566, 45.75, 43.2965],
    'longitude': [2.3522, 4.85, 5.3698],
    'type_poste': ['CDI', 'CDD', 'CDI']
})

# Préférences de l'utilisateur
ville_preferee = 'Paris'
type_entreprise_prefere = 'Start-up'
type_poste_desire = 'CDI'

# Coordonnées géographiques de la ville choisie par l'utilisateur
user_location = (48.8566, 2.3522)  # Paris (latitude, longitude)

# Calcul de la distance entre les villes
df['distance_en_km'] = df.apply(lambda row: geodesic(user_location, (row['latitude'], row['longitude'])).km, axis=1)

# Calcul de la similarité cosinus
vectorizer = TfidfVectorizer()
job_desc = df['intitule_poste'] + ' ' + df['type_entreprise'] + ' ' + df['type_poste']
user_preferences = ville_preferee + ' ' + type_entreprise_prefere + ' ' + type_poste_desire
X = vectorizer.fit_transform(job_desc.append(pd.Series(user_preferences)))
similarity_scores = cosine_similarity(X[:-1], X[-1])

# Normalisation de la distance
max_distance = df['distance_en_km'].max()
df['normalized_distance'] = 1 - (df['distance_en_km'] / max_distance)

# Attribution des poids aux similarités et à la distance
similarity_weight = 0.8
distance_weight = 0.2

# Création d'une nouvelle colonne pour stocker les scores de similarité
df['similarity_score'] = similarity_scores.flatten()

# Calcul de la similarité pondérée
df['weighted_similarity'] = df['similarity_score'] * similarity_weight
df['weighted_distance'] = df['normalized_distance'] * distance_weight
df['weighted_similarity_distance'] = df['weighted_similarity'] + df['weighted_distance']

# Tri des offres d'emploi par similarité pondérée et distance croissantes
sorted_df = df.sort_values(by=['weighted_similarity_distance', 'distance_en_km'], ascending=[False, True])

# Affichage des résultats
top_results = sorted_df.head(5)  # Vous pouvez ajuster le nombre de résultats à afficher
top_results[['intitule_poste', 'nom_entreprise', 'distance_en_km', 'weighted_similarity_distance']].head()


0          Développeur web Start-up CDI
1    Data analyst Grande entreprise CDD
2       Ingénieur logiciel Start-up CDI
0                    Paris Start-up CDI
dtype: object
       intitule_poste   nom_entreprise  distance_en_km  \
0     Développeur web      XYZ Company              10   
2  Ingénieur logiciel      XYZ Company              15   
1        Data analyst  ABC Corporation               5   

   weighted_similarity_distance  
0                      0.432068  
2                      0.365402  
1                      0.133333  


C:\Users\gaspa\AppData\Local\Temp\ipykernel_22188\615169175.py:27: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  X = vectorizer.fit_transform(job_desc.append(pd.Series(user_preferences)))
C:\Users\gaspa\AppData\Local\Temp\ipykernel_22188\615169175.py:30: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  print(job_desc.append(pd.Series(user_preferences)))
